## Calculate abundances with new decoder and FactorDis

In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.autograd.functional import jacobian

import torch.autograd as autograd
from torch.autograd import Variable

import numpy as np

from tagging.paths import path_dataset
from tagging.src.datasets import ApogeeDataset
from tagging.src.networks import ConditioningAutoencoder,Embedding_Decoder,Feedforward,ParallelDecoder, Autoencoder,Discriminator
import pandas as pd
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cuda = True if torch.cuda.is_available() else False


In [ ]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy()

In [ ]:
! pip install --upgrade torch 

In [ ]:
! pip install --upgrade torchvision

## Toy Dataset

Because of the long run time, I will be working on a toy dataset for now.

Here is the structure of the dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader


import matplotlib.pyplot as plt

In [ ]:
n_datapoints = 100000
n_signals = 10
n_bins = 200
fraction_signal = 0.05

In [ ]:
class ToyDataset(Dataset):
    def __init__(self,n_datapoints,n_signals,n_bins,fraction_signal):
        """
        n_datapoints: int
            number of datapoints in dataset
        n_signals: int
            number of laten variables to include in the signal
        n_bins: int
            number of bins in the latent signal
        fraction_signal: int
            fractions of bins containing each signal"""
            
        self.n_datapoints = n_datapoints
        self.n_signals = n_signals
        self.n_bins = n_bins
        self.fraction_signal = fraction_signal
    
        self.basis = self.generate_signal_basis(self.n_signals,self.n_bins,self.fraction_signal)
        self.amplitudes = self.generate_amplitudes_dataset(self.n_bins)
        self.dataset = self.generate_dataset(self.basis,self.amplitudes)
        self.dataset = torch.FloatTensor(self.dataset)

        
    def generate_signal_basis(self,n_signals,n_bins,fraction_signal):
        """returns an np.array of shape n_signals*n_bins where every row is the basis of its respective signal"""
        return (np.random.random_sample(size=(n_signals,n_bins))<fraction_signal).astype(float)
    
    def generate_amplitudes_dataset(self,n_bins):
        """generate the amplitude of each observation signal bin
        returns an array of shape n_signals*n_datapoints"""
        return np.random.normal(0,0.3,size= (n_signals,n_datapoints))
    
    def generate_dataset(self,basis,amplitudes):
        print(basis.shape)
        print(amplitudes.shape)
        return np.matmul(amplitudes.T,basis) 
    
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self,idx):
        return self.dataset[idx]
        
    

    

In [ ]:
dataset = ToyDataset(n_datapoints=n_datapoints,n_signals=n_signals,n_bins=n_bins,fraction_signal=fraction_signal)

In [ ]:
plt.plot(dataset.dataset[0])
plt.plot(dataset.dataset[1])
plt.plot(dataset.dataset[2])
plt.plot(dataset.dataset[3])
plt.plot(dataset.dataset[4])
plt.xlim(0,50)


plt.show()

## Pytorch setup

We generate our neural network and setup our optimizers

In [ ]:
n_z = n_signals
n_hidden = 10
lr = 0.001
n_batch = 64

In [ ]:
encoder = Feedforward([n_bins,128,32,n_z],activation=nn.LeakyReLU()).to(device)
decoder = ParallelDecoder(n_bins=n_bins,n_hidden=n_hidden,n_latent=n_z,activation=nn.SELU()).to(device)
autoencoder = Autoencoder(encoder,decoder,n_bins=n_bins).to(device)
optimizer_autoencoder = torch.optim.Adam(autoencoder.parameters(), lr=lr)

In [ ]:
loader = torch.utils.data.DataLoader(dataset = dataset,
                                     batch_size = n_batch,
                                     shuffle= True,
                                     drop_last=True)

In [ ]:
loss = nn.MSELoss()


## Train vanilla autoencoder

In [ ]:
for i in range(20000):

    for j,(x) in enumerate(loader):

        optimizer_autoencoder.zero_grad()
        x_pred,z = autoencoder(x.to(device))

        err_pred = loss(x_pred,x.to(device))  
        err_tot = err_pred

        err_tot.backward()
        optimizer_autoencoder.step()
        if j%100==0:
            print(f"err:{err_tot}")


## Train adverserialautoencoder

We add a loss term to ensure the latent is gaussian.

In [ ]:
encoder = Feedforward([n_bins,128,32,n_z],activation=nn.LeakyReLU()).to(device)
decoder = ParallelDecoder(n_bins=n_bins,n_hidden=n_hidden,n_latent=n_z,activation=nn.SELU()).to(device)
autoencoder = Autoencoder(encoder,decoder,n_bins=n_bins).to(device)
optimizer_G = torch.optim.Adam(autoencoder.parameters(), lr=lr)

In [ ]:
discriminator = Feedforward([n_z,256,128,32,1],activation=nn.SELU()).to(device)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr)
lambda_gp = 10
n_critic = 10
lambda_fact = 10**-3

In [ ]:
def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = torch.FloatTensor(np.random.random((real_samples.size(0), 1))).to(device)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = torch.FloatTensor(real_samples.shape[0], 1).fill_(1.0).to(device)
    fake.requires_grad=False
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [ ]:
batches_done = 0
for epoch in range(20000):

    for j,(x) in enumerate(loader):

        z_real = torch.randn(n_batch, n_z).to(device)

        optimizer_G.zero_grad()
        
        x_pred,z_fake = autoencoder(x.to(device))
        
        
        optimizer_D.zero_grad()
        real_validity = discriminator(z_real)
        fake_validity = discriminator(z_fake)
        
        gradient_penalty = compute_gradient_penalty(discriminator, z_real.data, z_fake.data)
        
        d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty
        d_loss.backward(retain_graph=True)
        optimizer_D.step()

        optimizer_G.zero_grad()

        # Train the generator every n_critic steps
        if i % n_critic == 0:

            fake_validity = discriminator(z_fake)
            real_validity = discriminator(z_real)

            err_pred = loss(x_pred,x.to(device))

            g_loss = err_pred-lambda_fact*(torch.mean(fake_validity)-torch.mean(real_validity))
            g_loss.backward()
            optimizer_G.step()
            if i%100*n_critic ==0:
                print(
                    "[Epoch %d/%d] [Batch %d/%d] [D: %f] [G: %f] [R: %f]"
                    % (epoch, 100, i, len(loader), d_loss.item(), torch.mean(fake_validity).item(),err_pred.item())
                )

        
                batches_done += n_critic



In [ ]:
to_numpy(z_real).flatten().shape

In [ ]:
plt.hist(to_numpy(z_real).flatten(),alpha=0.5)
plt.hist(to_numpy(z_fake).flatten(),alpha=0.5)

In [ ]:
plt.hist(to_numpy(z_real).flatten(),alpha=0.5)
plt.hist(to_numpy(z_fake).flatten(),alpha=0.5)

In [ ]:
z_real.shape

## Sparsify latent

We sparsify our autoencoders dependence on the latent. We create a vector ```switch``` containing the fraction of noise to replace the signal with

In [ ]:
class ParallelSwitchDecoder(nn.Module):
    def __init__(self,n_bins=100,n_hidden = 10,n_latent=22,activation=nn.LeakyReLU()):
        super(ParallelSwitchDecoder, self).__init__()
        self.n_bins = n_bins
        self.n_hidden = n_hidden
        self.n_latent = n_latent
        self.n_output = 1
        self.activation = activation
        self.conv1 = nn.Conv1d(in_channels=self.n_latent * self.n_bins, out_channels= self.n_hidden * self.n_bins, kernel_size=1, groups=self.n_bins)
        self.conv2 = nn.Conv1d(in_channels=self.n_hidden * self.n_bins, out_channels= self.n_hidden * self.n_bins, kernel_size=1, groups=self.n_bins)
        self.conv3 = nn.Conv1d(in_channels=self.n_hidden * self.n_bins, out_channels= self.n_output * self.n_bins, kernel_size=1, groups=self.n_bins)
        
        
    def forward(self, latent, switch):
        #print(f"switch:{switch}")
        repeated_latent = latent.repeat(1,self.n_bins)
        noise = torch.randn(n_batch,n_bins*n_z).to(device)
        #print(repeated_latent.shape)
        #print(noise.shape)
        #print(switch.shape)

        mixed_latent = noise*switch.sigmoid()+switch.sigmoid()*repeated_latent
        mixed_latent = mixed_latent.unsqueeze(2)
        #print(f"repeated_latent:{repeated_latent.shape}")

        hidden1 = self.activation(self.conv1(mixed_latent))
        #print(f"hidden1:{hidden1.shape}")

        hidden2 = self.activation(self.conv2(hidden1))
        #print(f"hidden2:{hidden2.shape}")

        output = self.conv3(hidden2)
        #print(f"output:{output.shape}")
        output = torch.squeeze(output)
        return output

In [ ]:

class AutoencoderSwitch(nn.Module):
    def __init__(self,encoder,decoder,n_bins = None):
        super(AutoencoderSwitch, self).__init__()
        self.n_bins = n_bins
        self.encoder = encoder
        self.decoder = decoder

        
    def forward(self, x,switch,train_encoder=True,train_decoder=True):
        latent=None
        output=None
        if train_encoder:
            x= self.encoder(x)
            latent = x
        if train_decoder:
            x = self.decoder(x,switch)
            output = x
        return output,latent



In [ ]:
switch = torch.randn(n_batch,n_bins*n_z).to(device)
#switch = torch.sigmoid(switch)
switch.requires_grad=True
switch.shape
optimizer_switch = torch.optim.Adam([switch], lr=lr)

In [ ]:
encoder = Feedforward([n_bins,128,32,n_z],activation=nn.LeakyReLU()).to(device)
decoder = ParallelSwitchDecoder(n_bins=n_bins,n_hidden=n_hidden,n_latent=n_z,activation=nn.SELU()).to(device)
autoencoder = AutoencoderSwitch(encoder,decoder,n_bins=n_bins).to(device)
optimizer_G = torch.optim.Adam(autoencoder.parameters(), lr=lr)

In [ ]:
discriminator = Feedforward([n_z,256,128,32,1],activation=nn.SELU()).to(device)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr)
lambda_gp = 10
n_critic = 5
lambda_fact = 10**-3
lambda_switch = 0.0001

In [ ]:
batches_done = 0
for epoch in range(20000):

    for i,(x) in enumerate(loader):
        z_real = torch.randn(n_batch, n_z).to(device)

        optimizer_G.zero_grad()
        
        x_pred,z_fake = autoencoder(x.to(device),switch)
        
        
        optimizer_D.zero_grad()
        real_validity = discriminator(z_real)
        fake_validity = discriminator(z_fake)
        
        gradient_penalty = compute_gradient_penalty(discriminator, z_real.data, z_fake.data)
        
        d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty
        d_loss.backward(retain_graph=True)
        optimizer_D.step()

        optimizer_G.zero_grad()
        optimizer_switch.zero_grad()

        # Train the generator every n_critic steps
        if i % n_critic == 0:

            fake_validity = discriminator(z_fake)
            real_validity = discriminator(z_real)

            err_pred = loss(x_pred,x.to(device))
            err_switch = torch.mean(switch.sigmoid())

            
            g_loss = err_pred-lambda_fact*(torch.mean(fake_validity)-torch.mean(real_validity))+lambda_switch*err_switch
            g_loss.backward()
            optimizer_G.step()
            optimizer_switch.step()

            if i%10*n_critic ==0:
                print(
                    "[Epoch %d/%d] [Batch %d/%d] [D: %f] [G: %f] [R: %f] [S: %f]"
                    % (epoch, 100, i, len(loader), d_loss.item(), torch.mean(fake_validity).item(),err_pred.item(),err_switch.item())
                )
        
                batches_done += n_critic



In [ ]:
switch.sigmoid()[:,0]

## Sparsify with batchnorm

Instead of adding a discriminator that enforces gaussianity we use a batchnorm layer.



In [ ]:
class ParallelSwitchDecoder(nn.Module):
    def __init__(self,n_bins=100,n_hidden = 10,n_latent=22,activation=nn.LeakyReLU()):
        super(ParallelSwitchDecoder, self).__init__()
        self.n_bins = n_bins
        self.n_hidden = n_hidden
        self.n_latent = n_latent
        self.n_output = 1
        self.activation = activation
        self.conv1 = nn.Conv1d(in_channels=self.n_latent * self.n_bins, out_channels= self.n_hidden * self.n_bins, kernel_size=1, groups=self.n_bins)
        self.conv2 = nn.Conv1d(in_channels=self.n_hidden * self.n_bins, out_channels= self.n_hidden * self.n_bins, kernel_size=1, groups=self.n_bins)
        self.conv3 = nn.Conv1d(in_channels=self.n_hidden * self.n_bins, out_channels= self.n_output * self.n_bins, kernel_size=1, groups=self.n_bins)
        
        
    def forward(self, latent, switch):
        #print(f"switch:{switch}")
        repeated_latent = latent.repeat(1,self.n_bins)
        noise = torch.randn(n_batch,n_bins*n_z).to(device)
        #print(repeated_latent.shape)
        #print(noise.shape)
        #print(switch.shape)

        mixed_latent = noise*switch.sigmoid()+switch.sigmoid()*repeated_latent
        mixed_latent = mixed_latent.unsqueeze(2)
        #print(f"repeated_latent:{repeated_latent.shape}")

        hidden1 = self.activation(self.conv1(mixed_latent))
        #print(f"hidden1:{hidden1.shape}")

        hidden2 = self.activation(self.conv2(hidden1))
        #print(f"hidden2:{hidden2.shape}")

        output = self.conv3(hidden2)
        #print(f"output:{output.shape}")
        output = torch.squeeze(output)
        return output

In [ ]:

class AutoencoderBatchNormSwitch(nn.Module):
    def __init__(self,encoder,decoder,n_bins = None):
        super(AutoencoderBatchNormSwitch, self).__init__()
        self.n_bins = n_bins
        self.encoder = encoder
        self.decoder = decoder
        self.batchnorm_layer = nn.BatchNorm1d(n_z)

        
    def forward(self, x,switch,train_encoder=True,train_decoder=True):
        latent=None
        output=None
        if train_encoder:
            x= self.encoder(x)
            x = self.batchnorm_layer(x)
            latent = x
        if train_decoder:
            x = self.decoder(x,switch)
            output = x
        return output,latent


In [ ]:
switch = torch.randn(n_batch,n_bins*n_z).to(device)
#switch = torch.sigmoid(switch)
switch.requires_grad=True
switch.shape
optimizer_switch = torch.optim.Adam([switch], lr=lr)

In [ ]:
encoder = Feedforward([n_bins,128,32,n_z],activation=nn.LeakyReLU()).to(device)
decoder = ParallelSwitchDecoder(n_bins=n_bins,n_hidden=n_hidden,n_latent=n_z,activation=nn.SELU()).to(device)
autoencoder = AutoencoderBatchNormSwitch(encoder,decoder,n_bins=n_bins).to(device)
optimizer_autoencoder = torch.optim.Adam(autoencoder.parameters(), lr=lr)

In [ ]:
lambda_switch = 0.05

In [ ]:
for i in range(20000):

    for j,(x) in enumerate(loader):

        optimizer_autoencoder.zero_grad()
        optimizer_switch.zero_grad()

        x_pred,z = autoencoder(x.to(device),switch)

        err_rec = loss(x_pred,x.to(device))
        err_switch = torch.mean(switch.sigmoid())

        err_tot = err_rec+lambda_switch

        err_tot.backward()
        optimizer_autoencoder.step()
        optimizer_switch.step()

        if j%100==0:
            print(f"err_tot:{err_tot},err_rec:{err_rec},err_switch:{err_switch}")
            

## Jacobian Training

Here we present a method that uses the jacobian to train this neural network

In [ ]:
encoder = Feedforward([n_bins,128,32,n_z],activation=nn.LeakyReLU()).to(device)
decoder = Feedforward([n_z,32,128,n_bins],activation=nn.LeakyReLU()).to(device)
autoencoder = Autoencoder(encoder,decoder,n_bins=n_bins).to(device)
optimizer_autoencoder = torch.optim.Adam(autoencoder.parameters(), lr=lr)
optimizer_encoder = torch.optim.Adam(encoder.parameters(), lr=lr)
optimizer_decoder = torch.optim.Adam(decoder.parameters(), lr=lr)

In [ ]:
loader = torch.utils.data.DataLoader(dataset = dataset,
                                     batch_size = n_batch,
                                     shuffle= True,
                                     drop_last=True)

In [ ]:
def average_batch_decoder(z):
    x_pred = decoder(z)
    x_flatened = torch.mean(x_pred,dim=0)
    return x_flatened

def loss_jacobian(z):
    #print(z.shape)
    jac_z = jacobian(average_batch_decoder, z, create_graph=True)
    #print(jac_z.shape)
    jac_z_unbatched = torch.sum(jac_z,dim=1)
    normalized_jac = torch.abs(jac_z_unbatched)/torch.sum(torch.abs(jac_z_unbatched),dim=1).unsqueeze(1).repeat(1,z.shape[1])
    loss = torch.mean(torch.sum(normalized_jac**0.5,dim=1))
    return loss

In [ ]:
def average_batch_decoder(z):
    x_pred = decoder(z)
    x_flatened = torch.mean(x_pred,dim=0)
    return x_flatened

"""def loss_jacobian(z):
    #print(z.shape)
    jac_z = jacobian(average_batch_decoder, z, create_graph=True)
    #print(jac_z.shape)
    jac_z_unbatched = torch.sum(jac_z,dim=1)
    normalized_jac = torch.abs(jac_z_unbatched)/torch.sum(torch.abs(jac_z_unbatched),dim=1).unsqueeze(1).repeat(1,z.shape[1])
    loss = torch.mean(torch.sum(normalized_jac**0.5,dim=1))
    return loss
"""

def loss_jacobian(z):
    #print(z.shape)
    jac_z = jacobian(average_batch_decoder, z, create_graph=True)
    #print(jac_z.shape)
    jac_z_unbatched = torch.sum(jac_z,dim=1)
    #normalized_jac = torch.abs(jac_z_unbatched)/torch.sum(torch.abs(jac_z_unbatched),dim=1).unsqueeze(1).repeat(1,z.shape[1])
    normalized_jac = jac_z_unbatched/torch.norm(jac_z_unbatched,p=2,dim=1).unsqueeze(1)
    loss = torch.mean(torch.sum(torch.abs(normalized_jac),dim=1)) #stopped multiplying by 0.5
    
    return loss

In [ ]:
jac_z = jacobian(average_batch_decoder, z, create_graph=True)
jac_z_unbatched = torch.sum(jac_z,dim=1)
jac_z.shape

In [ ]:
torch.sum(torch.abs(jac_z_unbatched),dim=1)

In [ ]:
torch.norm(jac_z_unbatched,p=2)

In [ ]:
torch.abs(jac_z_unbatched)/torch.sum(torch.abs(jac_z_unbatched),dim=1).unsqueeze(1)

In [ ]:
err=0

In [ ]:
for epoch in range(500):
  for x in loader:
    optimizer_encoder.zero_grad()
    optimizer_decoder.zero_grad()
    z = encoder(x.to(device))
    x_pred = decoder(z)
    #err_jacobian = loss_jacobian(z)
    err_rec = recon_loss(x_pred,x.to(device))
    #err = err_jacobian
    err = err_rec#+0.0001*err_jacobian
    #err = err_rec
    err.backward()
    optimizer_decoder.step()
    optimizer_encoder.step()
    print("err_jac:{},err_rec:{}".format(err_jacobian,err_rec))

In [ ]:
to_numpy(x_pred).shape

In [ ]:
plt.plot(to_numpy(x_pred)[0],c="orange")
plt.plot(to_numpy(x)[0],c="red")
plt.plot(to_numpy(x_pred)[2],c="blue")
plt.plot(to_numpy(x)[2],c="purple")
plt.xlim(0,50)

In [ ]:
x = dataset[0]
z = encoder(dataset[0].unsqueeze(0).to(device))
x_pred1 = decoder(z)
z[0,0]=1
x_pred2 = decoder(z)
z[0,0]=-0.5
x_pred3 = decoder(z)

plt.plot(to_numpy(x_pred1[0]))
plt.plot(to_numpy(x_pred2[0]))
plt.plot(to_numpy(x_pred3[0]))
plt.plot(dataset.basis[0],label="basis")
plt.xlim(100,200)

In [ ]:
x = dataset[0]
z = encoder(dataset[0].unsqueeze(0).to(device))
x_pred1 = decoder(z)
z[0,9]=1
x_pred2 = decoder(z)
z[0,9]=-0.5
x_pred3 = decoder(z)

plt.plot(to_numpy(x_pred1[0]))
plt.plot(to_numpy(x_pred2[0]))
plt.plot(to_numpy(x_pred3[0]))
plt.plot(dataset.basis[0],label="basis")
plt.xlim(0,200)

In [ ]:
x = dataset[0]
z = encoder(dataset[0].unsqueeze(0).to(device))
i=9
x_pred1 = decoder(z)
z[0,i]=0.5
x_pred2 = decoder(z)
z[0,i]=-0.5
x_pred3 = decoder(z)

plt.plot(to_numpy(x_pred1[0]))
plt.plot(to_numpy(x_pred2[0]))
plt.plot(to_numpy(x_pred3[0]))
plt.plot(dataset.basis[1],label="basis")
plt.xlim(150,200)

### Visualizing abundances vs learned features

In [ ]:
def encode_dataset(loader,encoder):
    z_list = []
    for x in loader:
        z = encoder(x)

In [ ]:
z_list = []
for i in range(100):
    x = dataset[100*i:100*i+100]
    z = encoder(x.to(device))
    z_list.append(to_numpy(z))

In [ ]:
z_array = np.concatenate(z_list,axis=0)

In [ ]:
amplitudes =  dataset.amplitudes.T

In [ ]:
amplitudes[:,0].shape

In [ ]:
import matplotlib.gridspec as gridspec

fig = plt.figure()
gs = gridspec.GridSpec(10,10)
for n_amplitude in range(10):
    for n_z in range(10): 
        ax = fig.add_subplot(gs[n_amplitude, n_z])
        ax.scatter(amplitudes[:1000,n_amplitude],z_array[:1000,n_z])
        ax.set_axis_off()
        
fig.suptitle("latent vs \"abundance\"")

In [ ]:
import matplotlib.gridspec as gridspec

fig = plt.figure()
gs = gridspec.GridSpec(10,10)
for n_amplitude in range(10):
    for n_z in range(10): 
        ax = fig.add_subplot(gs[n_amplitude, n_z])
        ax.scatter(amplitudes[:1000,n_amplitude],z_array[:1000,n_z])
        ax.set_axis_off()
        
fig.suptitle("latent vs \"abundance\"")

In [ ]:
plt.scatter(amplitudes[:10000,0],z_array[:10000,0])

In [ ]:
plt.scatter(amplitudes[:10000,0],z_array[:10000,0])

In [ ]:
plt.scatter(amplitudes[:10000,0],z_array[:10000,0])

In [ ]:
plt.scatter(amplitudes[:10000,2],z_array[:10000,4])

In [ ]:
plt.scatter(amplitudes[:10000,9],z_array[:10000,4])

In [ ]:
np.sum(dataset.basis,axis=1)

In [ ]:
z = encoder(dataset[0].unsqueeze(0).to(device))

In [ ]:
z

In [ ]:
z = encoder(dataset[0].unsqueeze(0).to(device))
x = to_numpy(decoder(z))[0]
plt.plot(x)

z[0,0]=0.1
plt.plot(to_numpy(decoder(z))[0],label="0.1")
plt.plot()

z[0,0]=0.2
plt.plot(to_numpy(decoder(z))[0],label="0.2")
plt.plot()
plt.xlim(0,50)


z[0,0]=0.3
plt.plot(to_numpy(decoder(z))[0],label="0.3")
plt.plot(dataset.basis[i],label="basis")
plt.legend()
plt.xlim(0,100)

In [ ]:
z = encoder(dataset[0].unsqueeze(0).to(device))
x = to_numpy(decoder(z))[0]
plt.plot(x)

z[0,1]=0.1
plt.plot(to_numpy(decoder(z))[0],label="0.1")
plt.plot()

z[0,1]=0.2
plt.plot(to_numpy(decoder(z))[0],label="0.2")
plt.plot()
plt.xlim(0,50)


z[0,1]=0.3
plt.plot(to_numpy(decoder(z))[0],label="0.3")
plt.plot(dataset.basis[8],label="basis")
plt.legend()
plt.xlim(0,50)

In [ ]:
z = encoder(dataset[0].unsqueeze(0).to(device))
x = to_numpy(decoder(z))[0]
plt.plot(x)

z[0,2]=0.1
plt.plot(to_numpy(decoder(z))[0],label="0.1")
plt.plot()

z[0,2]=0.2
plt.plot(to_numpy(decoder(z))[0],label="0.2")
plt.plot()
plt.xlim(0,50)


z[0,2]=0.3
plt.plot(to_numpy(decoder(z))[0],label="0.3")
plt.plot(dataset.basis[3],label="basis")
plt.legend()
plt.xlim(100,200)

In [ ]:
for i in range(10):
    plt.plot(dataset.basis[i])
    plt.title(i)
    plt.xlim(0,50)
    plt.show()

    